# Week 11: Prediction and Interaction

### Topics Covered

- Modeling driver behavior; Predicting the trajectories of other agents; Game Theory and Reinforcement Learning for complex intersections

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon, FancyArrowPatch, Wedge
from matplotlib.animation import FuncAnimation
from scipy.stats import multivariate_normal
from scipy.optimize import minimize
import itertools

# Set random seed for reproducibility
np.random.seed(42)

## 1. Prediction Fundamentals

Predicting the future behavior of other agents (vehicles, pedestrians, cyclists) is crucial for safe autonomous driving.

### Why Prediction Matters

- **Safety**: Anticipate dangerous situations before they occur
- **Comfort**: Smooth driving requires anticipating other agents
- **Efficiency**: Optimize routes based on predicted traffic flow
- **Planning**: Generate ego trajectories that account for others' future states

### The Prediction Problem

**Given:**
- Observed history: $H = \{s_1, s_2, \ldots, s_t\}$ where $s_i = (x_i, y_i, v_i, \theta_i)$
- Context: Road geometry, traffic rules, map information

**Predict:**
- Future trajectory: $\hat{T} = \{s_{t+1}, s_{t+2}, \ldots, s_{t+T_h}\}$ over horizon $T_h$

### Prediction Challenges

1. **Uncertainty**: Multiple plausible futures
2. **Multimodality**: Agents can take different actions (turn left, go straight, etc.)
3. **Interaction**: Agents react to each other
4. **Long horizons**: Uncertainty grows with time
5. **Rare events**: Hard to learn from data

### Prediction Approaches

**1. Physics-Based Models**
- Constant Velocity (CV)
- Constant Acceleration (CA)
- Kinematic bicycle model

**2. Learning-Based Models**
- Recurrent Neural Networks (RNN/LSTM)
- Convolutional Social Pooling
- Graph Neural Networks
- Transformers (attention mechanisms)

**3. Hybrid Approaches**
- Combine physics models with learned intent prediction
- Physics-informed neural networks

**4. Probabilistic Models**
- Gaussian Processes
- Mixture Density Networks
- Conditional Variational Autoencoders (CVAE)

---

## 2. Constant Velocity and Constant Acceleration Models

### Constant Velocity Model

Assumes the agent maintains current velocity:

$$
\begin{align}
x_{t+\Delta t} &= x_t + v_x \Delta t \\
y_{t+\Delta t} &= y_t + v_y \Delta t \\
v_{x,t+\Delta t} &= v_x \\
v_{y,t+\Delta t} &= v_y
\end{align}
$$

**Pros**: Simple, computationally efficient  
**Cons**: Doesn't model turning or speed changes

### Constant Acceleration Model

Assumes constant acceleration:

$$
\begin{align}
x_{t+\Delta t} &= x_t + v_x \Delta t + \frac{1}{2} a_x \Delta t^2 \\
y_{t+\Delta t} &= y_t + v_y \Delta t + \frac{1}{2} a_y \Delta t^2 \\
v_{x,t+\Delta t} &= v_x + a_x \Delta t \\
v_{y,t+\Delta t} &= v_y + a_y \Delta t
\end{align}
$$

**Pros**: Models acceleration/braking  
**Cons**: Still doesn't capture complex maneuvers

### Uncertainty Propagation

Both models can be extended with uncertainty using Kalman filters:

$$
\begin{align}
\hat{s}_{t+1} &= F \hat{s}_t \\
\Sigma_{t+1} &= F \Sigma_t F^T + Q
\end{align}
$$

Where $\Sigma$ is the covariance matrix representing prediction uncertainty.

In [ ]:
# Implementation: Multi-Model Trajectory Prediction

class TrajectoryPredictor:
    """Predicts future trajectories using multiple motion models"""
    
    def __init__(self, dt=0.1):
        self.dt = dt
    
    def constant_velocity(self, state, horizon):
        """
        Predict using constant velocity model
        state: [x, y, vx, vy]
        horizon: number of time steps
        """
        x, y, vx, vy = state
        trajectory = [state]
        
        for _ in range(horizon):
            x = x + vx * self.dt
            y = y + vy * self.dt
            trajectory.append(np.array([x, y, vx, vy]))
        
        return np.array(trajectory)
    
    def constant_acceleration(self, state, accel, horizon):
        """
        Predict using constant acceleration model
        state: [x, y, vx, vy]
        accel: [ax, ay]
        horizon: number of time steps
        """
        x, y, vx, vy = state
        ax, ay = accel
        trajectory = [state]
        
        for _ in range(horizon):
            x = x + vx * self.dt + 0.5 * ax * self.dt**2
            y = y + vy * self.dt + 0.5 * ay * self.dt**2
            vx = vx + ax * self.dt
            vy = vy + ay * self.dt
            trajectory.append(np.array([x, y, vx, vy]))
        
        return np.array(trajectory)
    
    def constant_turn_rate(self, state, omega, horizon):
        """
        Predict using constant turn rate model
        state: [x, y, v, theta]
        omega: turn rate (rad/s)
        horizon: number of time steps
        """
        x, y, v, theta = state
        trajectory = [[x, y, v, theta]]
        
        for _ in range(horizon):
            if abs(omega) < 1e-6:  # Straight line
                x = x + v * np.cos(theta) * self.dt
                y = y + v * np.sin(theta) * self.dt
            else:  # Circular arc
                x = x + (v / omega) * (np.sin(theta + omega * self.dt) - np.sin(theta))
                y = y + (v / omega) * (-np.cos(theta + omega * self.dt) + np.cos(theta))
            theta = theta + omega * self.dt
            trajectory.append([x, y, v, theta])
        
        return np.array(trajectory)


# Demonstration: Compare prediction models
def demonstrate_prediction_models():
    predictor = TrajectoryPredictor(dt=0.1)
    horizon = 50
    
    # Initial state for CV/CA models: [x, y, vx, vy]
    state_linear = np.array([0.0, 0.0, 10.0, 5.0])  # Moving diagonally
    
    # Initial state for CTR model: [x, y, v, theta]
    speed = np.sqrt(10**2 + 5**2)
    heading = np.arctan2(5, 10)
    state_turn = np.array([0.0, 0.0, speed, heading])
    
    # Generate predictions
    cv_pred = predictor.constant_velocity(state_linear, horizon)
    ca_pred_accel = predictor.constant_acceleration(state_linear, accel=[1.0, 0.5], horizon=horizon)
    ca_pred_decel = predictor.constant_acceleration(state_linear, accel=[-1.0, -0.5], horizon=horizon)
    
    # Turn predictions
    ctr_left = predictor.constant_turn_rate(state_turn, omega=0.2, horizon=horizon)
    ctr_right = predictor.constant_turn_rate(state_turn, omega=-0.2, horizon=horizon)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Linear motion models
    ax1 = axes[0]
    ax1.plot(cv_pred[:, 0], cv_pred[:, 1], 'b-', linewidth=2, label='Constant Velocity', marker='o', markevery=10)
    ax1.plot(ca_pred_accel[:, 0], ca_pred_accel[:, 1], 'g-', linewidth=2, label='Constant Accel (+)', marker='s', markevery=10)
    ax1.plot(ca_pred_decel[:, 0], ca_pred_decel[:, 1], 'r-', linewidth=2, label='Constant Accel (-)', marker='^', markevery=10)
    ax1.plot(0, 0, 'ko', markersize=10, label='Start')
    ax1.set_xlabel('X Position (m)', fontsize=12)
    ax1.set_ylabel('Y Position (m)', fontsize=12)
    ax1.set_title('Linear Motion Models Comparison', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')
    
    # Turn rate model
    ax2 = axes[1]
    ax2.plot(ctr_left[:, 0], ctr_left[:, 1], 'b-', linewidth=2, label='Left Turn (ω=+0.2)', marker='o', markevery=10)
    ax2.plot(ctr_right[:, 0], ctr_right[:, 1], 'r-', linewidth=2, label='Right Turn (ω=-0.2)', marker='s', markevery=10)
    ax2.plot(0, 0, 'ko', markersize=10, label='Start')
    
    # Add velocity vectors at start
    ax2.arrow(0, 0, 5*np.cos(heading), 5*np.sin(heading), 
             head_width=2, head_length=1, fc='green', ec='green', alpha=0.6, label='Initial Velocity')
    
    ax2.set_xlabel('X Position (m)', fontsize=12)
    ax2.set_ylabel('Y Position (m)', fontsize=12)
    ax2.set_title('Constant Turn Rate Model', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    ax2.axis('equal')
    
    plt.tight_layout()
    plt.show()
    
    print("Prediction Models Comparison:")
    print(f"  Time horizon: {horizon * predictor.dt:.1f} seconds")
    print(f"  CV final position: ({cv_pred[-1, 0]:.1f}, {cv_pred[-1, 1]:.1f}) m")
    print(f"  CA(+) final position: ({ca_pred_accel[-1, 0]:.1f}, {ca_pred_accel[-1, 1]:.1f}) m")
    print(f"  CA(-) final position: ({ca_pred_decel[-1, 0]:.1f}, {ca_pred_decel[-1, 1]:.1f}) m")

demonstrate_prediction_models()

---

## 3. Game Theory for Interaction Modeling

Game theory provides a mathematical framework for modeling strategic interactions between multiple agents.

### Why Game Theory for Autonomous Driving?

- **Strategic Behavior**: Drivers make decisions considering what others will do
- **Multi-Agent**: Multiple vehicles interact simultaneously
- **Incomplete Information**: Drivers don't know others' exact intentions
- **Sequential Decisions**: Actions unfold over time

### Key Game Theory Concepts

**1. Players**: Autonomous vehicle (ego) and other agents

**2. Actions**: Discrete choices (yield, go, merge) or continuous (acceleration, steering)

**3. Payoffs**: Utility functions representing goals:
- Safety: Avoid collisions
- Efficiency: Minimize travel time
- Comfort: Smooth acceleration

**4. Nash Equilibrium**: A state where no player can improve their payoff by unilaterally changing strategy

### Types of Games in Driving

**1. Simultaneous Games** (Matrix Form)
- Both players act at the same time
- Example: Intersection crossing

**2. Sequential Games** (Game Trees)
- Players take turns
- Example: Merge negotiation

**3. Repeated Games**
- Same interaction happens multiple times
- Example: Highway lane changes

**4. Differential Games**
- Continuous-time, continuous-action
- Example: Trajectory planning with interaction

---

## 3.1 Intersection Game Example

Consider a 4-way intersection with two vehicles approaching:

**Players**: Vehicle A (ego), Vehicle B (other)

**Actions**: 
- Go (G): Proceed through intersection
- Yield (Y): Wait for other vehicle

**Payoffs** (Vehicle A, Vehicle B):

$$
\begin{array}{c|c|c}
 & \text{B: Go} & \text{B: Yield} \\
\hline
\text{A: Go} & (-100, -100) & (10, -5) \\
\text{A: Yield} & (-5, 10) & (-2, -2)
\end{array}
$$

- Both Go: Collision! Very negative payoff
- A Go, B Yield: A gets through quickly (+10), B waits (-5)
- A Yield, B Go: B gets through quickly (+10), A waits (-5)
- Both Yield: Both waste time (-2 each)

**Nash Equilibria**: (Go, Yield) and (Yield, Go)

The challenge: How to coordinate which equilibrium to select?

In [ ]:
# Implementation: Game Theoretic Intersection Model

class IntersectionGame:
    """Models intersection crossing as a 2-player game"""
    
    def __init__(self):
        # Payoff matrix: [player_A_action, player_B_action] -> (payoff_A, payoff_B)
        # Actions: 0 = Yield, 1 = Go
        self.payoff_matrix = np.array([
            [(-2, -2), (-5, 10)],   # A: Yield, B: Yield/Go
            [(10, -5), (-100, -100)] # A: Go, B: Yield/Go
        ])
    
    def get_payoff(self, action_a, action_b):
        """Get payoffs for both players given their actions"""
        return self.payoff_matrix[action_a, action_b]
    
    def find_nash_equilibria(self):
        """Find all pure strategy Nash equilibria"""
        equilibria = []
        
        for action_a in range(2):
            for action_b in range(2):
                is_equilibrium = True
                current_payoff = self.get_payoff(action_a, action_b)
                
                # Check if A can improve by deviating
                for alt_action_a in range(2):
                    if alt_action_a != action_a:
                        alt_payoff = self.get_payoff(alt_action_a, action_b)
                        if alt_payoff[0] > current_payoff[0]:
                            is_equilibrium = False
                            break
                
                # Check if B can improve by deviating
                for alt_action_b in range(2):
                    if alt_action_b != action_b:
                        alt_payoff = self.get_payoff(action_a, alt_action_b)
                        if alt_payoff[1] > current_payoff[1]:
                            is_equilibrium = False
                            break
                
                if is_equilibrium:
                    equilibria.append((action_a, action_b, current_payoff))
        
        return equilibria
    
    def best_response(self, other_action, player='A'):
        """Find best response to other player's action"""
        best_action = None
        best_payoff = -np.inf
        
        if player == 'A':
            for action in range(2):
                payoff = self.get_payoff(action, other_action)[0]
                if payoff > best_payoff:
                    best_payoff = payoff
                    best_action = action
        else:  # player == 'B'
            for action in range(2):
                payoff = self.get_payoff(other_action, action)[1]
                if payoff > best_payoff:
                    best_payoff = payoff
                    best_action = action
        
        return best_action, best_payoff


# Demonstration: Analyze intersection game
def demonstrate_intersection_game():
    game = IntersectionGame()
    
    print("=== Intersection Game Analysis ===\n")
    
    # Display payoff matrix
    print("Payoff Matrix (Player A, Player B):")
    print("           B: Yield    B: Go")
    print(f"A: Yield   {game.payoff_matrix[0,0]}   {game.payoff_matrix[0,1]}")
    print(f"A: Go      {game.payoff_matrix[1,0]}   {game.payoff_matrix[1,1]}")
    print()
    
    # Find Nash equilibria
    equilibria = game.find_nash_equilibria()
    print(f"Nash Equilibria: {len(equilibria)} found")
    action_names = ['Yield', 'Go']
    for eq in equilibria:
        action_a, action_b, payoffs = eq
        print(f"  ({action_names[action_a]}, {action_names[action_b]}): Payoffs = {payoffs}")
    print()
    
    # Best responses
    print("Best Response Analysis:")
    for action_b in range(2):
        br_a, payoff_a = game.best_response(action_b, player='A')
        print(f"  If B plays {action_names[action_b]}, A's best response: {action_names[br_a]} (payoff: {payoff_a})")
    
    for action_a in range(2):
        br_b, payoff_b = game.best_response(action_a, player='B')
        print(f"  If A plays {action_names[action_a]}, B's best response: {action_names[br_b]} (payoff: {payoff_b})")
    
    # Visualize game tree
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    
    # Title
    ax.text(5, 9.5, 'Intersection Game Tree', fontsize=16, fontweight='bold', ha='center')
    
    # Root
    ax.plot(5, 8, 'ko', markersize=15)
    ax.text(5, 8.3, 'Start', fontsize=11, ha='center', fontweight='bold')
    
    # Player A's decisions
    ax.plot([5, 2.5], [8, 6], 'k-', linewidth=2)
    ax.plot([5, 7.5], [8, 6], 'k-', linewidth=2)
    ax.text(3.5, 7.2, 'A: Yield', fontsize=10, ha='center', style='italic')
    ax.text(6.5, 7.2, 'A: Go', fontsize=10, ha='center', style='italic')
    
    ax.plot(2.5, 6, 'bo', markersize=12)
    ax.plot(7.5, 6, 'bo', markersize=12)
    
    # Player B's responses to A: Yield
    ax.plot([2.5, 1.5], [6, 4], 'b--', linewidth=1.5)
    ax.plot([2.5, 3.5], [6, 4], 'b--', linewidth=1.5)
    ax.text(1.8, 5.1, 'B: Yield', fontsize=9, ha='center', color='blue')
    ax.text(3.2, 5.1, 'B: Go', fontsize=9, ha='center', color='blue')
    
    # Player B's responses to A: Go
    ax.plot([7.5, 6.5], [6, 4], 'b--', linewidth=1.5)
    ax.plot([7.5, 8.5], [6, 4], 'b--', linewidth=1.5)
    ax.text(6.8, 5.1, 'B: Yield', fontsize=9, ha='center', color='blue')
    ax.text(8.2, 5.1, 'B: Go', fontsize=9, ha='center', color='blue')
    
    # Outcomes
    outcomes = [
        (1.5, 4, '(-2, -2)', 'Both wait', 'yellow'),
        (3.5, 4, '(-5, 10)', 'B goes', 'lightgreen'),
        (6.5, 4, '(10, -5)', 'A goes', 'lightgreen'),
        (8.5, 4, '(-100, -100)', 'CRASH!', 'red')
    ]
    
    for x, y, payoff, desc, color in outcomes:
        rect = Rectangle((x-0.6, y-0.3), 1.2, 0.6, facecolor=color, edgecolor='black', linewidth=2, alpha=0.7)
        ax.add_patch(rect)
        ax.text(x, y+0.05, payoff, fontsize=10, ha='center', fontweight='bold')
        ax.text(x, y-0.7, desc, fontsize=9, ha='center', style='italic')
    
    # Add Nash equilibria markers
    ax.text(3.5, 3.2, '★ Nash Eq.', fontsize=9, ha='center', color='green', fontweight='bold')
    ax.text(6.5, 3.2, '★ Nash Eq.', fontsize=9, ha='center', color='green', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

demonstrate_intersection_game()

---

## 4. Reinforcement Learning for Interactive Driving

Reinforcement Learning (RL) enables agents to learn optimal behavior through trial-and-error interaction with the environment.

### Why RL for Autonomous Driving Interactions?

- **Complex Scenarios**: Hard to manually specify rules for all situations
- **Learning from Experience**: Improve over time through data
- **Multi-Agent**: Can model interactions between multiple learners
- **Delayed Rewards**: Actions have long-term consequences

### RL Formulation

**Markov Decision Process (MDP)**:
- **State** $s_t$: Vehicle positions, velocities, road geometry
- **Action** $a_t$: Acceleration, steering angle
- **Reward** $r_t$: Scalar feedback (positive for good, negative for bad)
- **Transition** $p(s_{t+1}|s_t, a_t)$: Dynamics of environment
- **Policy** $\pi(a|s)$: Strategy mapping states to actions

**Objective**: Learn policy $\pi^*$ that maximizes expected cumulative reward:
$$
J(\pi) = \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r_t \right]
$$

Where $\gamma \in [0,1]$ is the discount factor.

### Q-Learning Algorithm

**Q-function** represents expected return starting from state $s$, taking action $a$:
$$
Q^*(s, a) = \max_{\pi} \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r_t \mid s_0=s, a_0=a, \pi \right]
$$

**Q-Learning Update Rule**:
$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_t + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t, a_t) \right]
$$

Where:
- $\alpha$: Learning rate
- $r_t + \gamma \max_{a'} Q(s_{t+1}, a')$: TD target
- $r_t + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t, a_t)$: TD error

### Multi-Agent RL for Intersection

In multi-agent settings, each vehicle learns simultaneously:
- **Independent Q-Learning**: Each agent learns ignoring others
- **Joint Action Learning**: Learn Q-values for joint actions
- **Actor-Critic Methods**: Learn policy and value function
- **Multi-Agent Deep RL**: Use neural networks for complex states

### Reward Design for Driving

Critical for learning safe and efficient behavior:

**Safety Rewards**:
- Large negative reward for collision: $r_{collision} = -1000$
- Penalty for getting too close: $r_{proximity} = -50 \cdot \max(0, d_{safe} - d)$

**Efficiency Rewards**:
- Reward for progress toward goal: $r_{progress} = v \cdot \cos(\theta_{error})$
- Penalty for time: $r_{time} = -1$ per step

**Comfort Rewards**:
- Penalty for high acceleration: $r_{comfort} = -0.1 \cdot |a|$

**Total Reward**:
$$
r_t = r_{progress} + r_{proximity} + r_{comfort} + r_{collision} + r_{time}
$$

In [ ]:
# Implementation: Q-Learning for Simple Merging Scenario

class MergeEnvironment:
    """Simple 1D highway merge environment"""
    
    def __init__(self):
        self.length = 100  # Highway length
        self.goal = 90     # Goal position
        self.safe_distance = 5.0
        self.reset()
    
    def reset(self):
        """Reset environment to initial state"""
        self.ego_pos = 10.0
        self.ego_vel = 5.0
        self.other_pos = 40.0
        self.other_vel = 6.0
        self.time_step = 0
        self.max_steps = 100
        return self._get_state()
    
    def _get_state(self):
        """Discretize continuous state"""
        # Relative position bins
        rel_pos = self.other_pos - self.ego_pos
        rel_pos_bin = min(max(int(rel_pos / 10), 0), 9)  # 10 bins
        
        # Relative velocity bins
        rel_vel = self.other_vel - self.ego_vel
        rel_vel_bin = min(max(int((rel_vel + 5) / 2), 0), 4)  # 5 bins
        
        # Distance to goal bins
        dist_to_goal = self.goal - self.ego_pos
        goal_bin = min(max(int(dist_to_goal / 20), 0), 4)  # 5 bins
        
        return (rel_pos_bin, rel_vel_bin, goal_bin)
    
    def step(self, action):
        """
        Execute action and return (next_state, reward, done)
        Actions: 0 = decelerate, 1 = maintain, 2 = accelerate
        """
        # Map action to acceleration
        accel_map = {0: -2.0, 1: 0.0, 2: 2.0}
        accel = accel_map[action]
        
        # Update ego vehicle
        self.ego_vel = np.clip(self.ego_vel + accel * 0.1, 0, 15)
        self.ego_pos += self.ego_vel * 0.1
        
        # Update other vehicle (constant velocity)
        self.other_pos += self.other_vel * 0.1
        
        self.time_step += 1
        
        # Calculate reward
        reward = self._calculate_reward()
        
        # Check termination conditions
        done = False
        if self.ego_pos >= self.goal:
            reward += 100  # Bonus for reaching goal
            done = True
        elif self.time_step >= self.max_steps:
            done = True
        elif abs(self.ego_pos - self.other_pos) < self.safe_distance:
            reward = -1000  # Collision penalty
            done = True
        
        next_state = self._get_state()
        
        return next_state, reward, done
    
    def _calculate_reward(self):
        """Calculate reward for current state"""
        reward = 0.0
        
        # Progress reward
        reward += self.ego_vel * 0.1
        
        # Time penalty
        reward -= 1.0
        
        # Proximity penalty
        distance = abs(self.ego_pos - self.other_pos)
        if distance < self.safe_distance * 2:
            reward -= 10 * (self.safe_distance * 2 - distance)
        
        # Comfort penalty (high acceleration)
        # (already implicitly handled by velocity changes)
        
        return reward


class QLearningAgent:
    """Q-Learning agent for discrete state-action spaces"""
    
    def __init__(self, n_states, n_actions, alpha=0.1, gamma=0.95, epsilon=0.1):
        self.n_states = n_states
        self.n_actions = n_actions
        self.alpha = alpha        # Learning rate
        self.gamma = gamma        # Discount factor
        self.epsilon = epsilon    # Exploration rate
        
        # Initialize Q-table
        self.Q = np.zeros(n_states + (n_actions,))
    
    def get_action(self, state, training=True):
        """Epsilon-greedy action selection"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)  # Explore
        else:
            return np.argmax(self.Q[state])  # Exploit
    
    def update(self, state, action, reward, next_state, done):
        """Q-learning update"""
        if done:
            td_target = reward
        else:
            td_target = reward + self.gamma * np.max(self.Q[next_state])
        
        td_error = td_target - self.Q[state + (action,)]
        self.Q[state + (action,)] += self.alpha * td_error


# Training loop
def train_q_learning(episodes=500):
    env = MergeEnvironment()
    agent = QLearningAgent(
        n_states=(10, 5, 5),  # State space dimensions
        n_actions=3,           # Action space size
        alpha=0.1,
        gamma=0.95,
        epsilon=0.2
    )
    
    episode_rewards = []
    episode_lengths = []
    
    print("Training Q-Learning agent...")
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False
        steps = 0
        
        while not done:
            action = agent.get_action(state, training=True)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            steps += 1
        
        episode_rewards.append(total_reward)
        episode_lengths.append(steps)
        
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            avg_length = np.mean(episode_lengths[-100:])
            print(f"  Episode {episode+1}/{episodes}: Avg Reward = {avg_reward:.2f}, Avg Length = {avg_length:.1f}")
    
    # Visualize learning progress
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot rewards
    ax1 = axes[0]
    window = 50
    smoothed_rewards = np.convolve(episode_rewards, np.ones(window)/window, mode='valid')
    ax1.plot(smoothed_rewards, linewidth=2, color='blue')
    ax1.set_xlabel('Episode', fontsize=12)
    ax1.set_ylabel('Total Reward (smoothed)', fontsize=12)
    ax1.set_title('Learning Curve: Cumulative Reward', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Plot episode lengths
    ax2 = axes[1]
    smoothed_lengths = np.convolve(episode_lengths, np.ones(window)/window, mode='valid')
    ax2.plot(smoothed_lengths, linewidth=2, color='green')
    ax2.set_xlabel('Episode', fontsize=12)
    ax2.set_ylabel('Episode Length (smoothed)', fontsize=12)
    ax2.set_title('Learning Curve: Episode Duration', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return agent, env

# Train and visualize
trained_agent, env = train_q_learning(episodes=500)

print("\nFinal Q-Learning Performance:")
print(f"  Q-table shape: {trained_agent.Q.shape}")
print(f"  Non-zero Q-values: {np.count_nonzero(trained_agent.Q)}/{trained_agent.Q.size}")

---

## Exercises

### Exercise 1: Multi-Modal Trajectory Prediction

**Objective:** Implement a probabilistic prediction system that outputs multiple trajectory hypotheses.

**Task:** Create a predictor that generates multiple plausible future trajectories for a vehicle at an intersection.

**Instructions:**
- Use the constant velocity and constant turn rate models to generate multiple hypotheses
- Assign probabilities to each hypothesis based on context (e.g., lane geometry, turn signals)
- Implement a maneuver classifier that predicts: {go straight, turn left, turn right}
- Visualize the predicted trajectory distribution as a probability cone

**Bonus:** Add uncertainty quantification using covariance ellipses at future time steps.

### Exercise 2: Mixed Strategy Nash Equilibrium

**Objective:** Find and analyze mixed strategy equilibria for the intersection game.

**Task:** Extend the `IntersectionGame` class to compute mixed strategy Nash equilibria.

**Background:** In a mixed strategy, players randomize their actions. For the intersection game, each player chooses a probability distribution over {Yield, Go}.

**Instructions:**
- Implement a function to find mixed strategy equilibria
- For each player, the expected payoff from both pure strategies must be equal when mixing optimally
- Compute the equilibrium probabilities $(p_A, p_B)$ where:
  - $p_A$ = probability that A plays "Go"
  - $p_B$ = probability that B plays "Go"
- Visualize the payoff landscape as a heatmap
- Compare pure vs. mixed equilibria in terms of expected payoffs

### Exercise 3: Deep Q-Learning for Lane Changing

**Objective:** Implement Deep Q-Network (DQN) for a more complex highway lane changing scenario.

**Task:** Extend the Q-learning implementation to use a neural network function approximator instead of a Q-table.

**Instructions:**
- Create a 2D highway environment with multiple lanes
- Use a neural network to approximate Q(s,a) for continuous state spaces
- Implement experience replay buffer
- Add target network for stable learning
- Train the agent to perform safe lane changes
- Compare performance with tabular Q-learning

**State representation:** [ego_position, ego_velocity, lane_id, nearby_vehicles_positions, nearby_vehicles_velocities]

**Actions:** {lane_left, maintain_lane, lane_right, accelerate, decelerate}

In [ ]:
# Exercise Solutions

# Exercise 1: Multi-Modal Trajectory Prediction
# TODO: Implement multi-modal trajectory prediction
#
# Suggested approach:
# 1. Define maneuver classes (straight, left turn, right turn)
# 2. For each maneuver, generate trajectory using appropriate motion model
# 3. Assign prior probabilities based on context
# 4. Visualize all hypotheses with transparency proportional to probability
#
# Example structure:
# class MultiModalPredictor:
#     def __init__(self, predictor):
#         self.predictor = predictor
#         self.maneuvers = ['straight', 'left', 'right']
#     
#     def predict_multimodal(self, state, context):
#         hypotheses = []
#         probabilities = []
#         
#         # Generate hypothesis for going straight
#         traj_straight = self.predictor.constant_velocity(state, horizon=50)
#         hypotheses.append(('straight', traj_straight))
#         
#         # Generate hypothesis for left turn
#         traj_left = self.predictor.constant_turn_rate(
#             convert_to_polar(state), omega=0.3, horizon=50
#         )
#         hypotheses.append(('left', traj_left))
#         
#         # Generate hypothesis for right turn
#         traj_right = self.predictor.constant_turn_rate(
#             convert_to_polar(state), omega=-0.3, horizon=50
#         )
#         hypotheses.append(('right', traj_right))
#         
#         # Compute probabilities based on context
#         probabilities = self.compute_maneuver_probabilities(state, context)
#         
#         return hypotheses, probabilities
#     
#     def compute_maneuver_probabilities(self, state, context):
#         # Use lane geometry, turn signals, traffic rules
#         # Simple example: uniform prior if no context
#         return [0.6, 0.2, 0.2]  # [straight, left, right]


# Exercise 2: Mixed Strategy Nash Equilibrium
# TODO: Implement mixed strategy equilibrium finder
#
# Suggested approach:
# 1. Extend IntersectionGame class
# 2. Set up indifference equations for each player
# 3. Solve for mixing probabilities
# 4. Verify equilibrium conditions
#
# Example structure:
# class IntersectionGameExtended(IntersectionGame):
#     def find_mixed_strategy_equilibrium(self):
#         """
#         For 2x2 game, player A is indifferent when:
#         p_B * U_A(Go, Go) + (1-p_B) * U_A(Go, Yield) = 
#         p_B * U_A(Yield, Go) + (1-p_B) * U_A(Yield, Yield)
#         
#         Solve for p_B, then similarly for p_A
#         """
#         # Player A's payoffs
#         u_a_go_go = self.payoff_matrix[1, 1][0]
#         u_a_go_yield = self.payoff_matrix[1, 0][0]
#         u_a_yield_go = self.payoff_matrix[0, 1][0]
#         u_a_yield_yield = self.payoff_matrix[0, 0][0]
#         
#         # Solve for p_B (probability B plays Go)
#         # p_B * u_a_go_go + (1-p_B) * u_a_go_yield = 
#         # p_B * u_a_yield_go + (1-p_B) * u_a_yield_yield
#         numerator = u_a_yield_yield - u_a_go_yield
#         denominator = (u_a_go_go - u_a_yield_go) - (u_a_go_yield - u_a_yield_yield)
#         p_B = numerator / denominator if denominator != 0 else 0.5
#         
#         # Similarly solve for p_A
#         u_b_go_go = self.payoff_matrix[1, 1][1]
#         u_b_yield_go = self.payoff_matrix[0, 1][1]
#         u_b_go_yield = self.payoff_matrix[1, 0][1]
#         u_b_yield_yield = self.payoff_matrix[0, 0][1]
#         
#         numerator = u_b_yield_yield - u_b_yield_go
#         denominator = (u_b_go_go - u_b_go_yield) - (u_b_yield_go - u_b_yield_yield)
#         p_A = numerator / denominator if denominator != 0 else 0.5
#         
#         return p_A, p_B
#     
#     def expected_payoff_mixed(self, p_A, p_B):
#         """Compute expected payoffs under mixed strategy"""
#         # E[U_A] = sum over all action pairs: p(a_A) * p(a_B) * U_A(a_A, a_B)
#         exp_payoff_A = 0
#         exp_payoff_B = 0
#         
#         for a_A in range(2):
#             for a_B in range(2):
#                 prob_joint = (p_A if a_A == 1 else 1-p_A) * (p_B if a_B == 1 else 1-p_B)
#                 payoffs = self.get_payoff(a_A, a_B)
#                 exp_payoff_A += prob_joint * payoffs[0]
#                 exp_payoff_B += prob_joint * payoffs[1]
#         
#         return exp_payoff_A, exp_payoff_B


# Exercise 3: Deep Q-Learning for Lane Changing
# TODO: Implement DQN with neural network
#
# Suggested approach:
# 1. Define neural network architecture (e.g., 2 hidden layers)
# 2. Implement experience replay buffer
# 3. Create target network (copy of Q-network, updated periodically)
# 4. Implement DQN training loop
# 5. Compare with tabular Q-learning
#
# Example structure (requires PyTorch or TensorFlow):
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from collections import deque
# import random
#
# class QNetwork(nn.Module):
#     def __init__(self, state_dim, action_dim, hidden_dim=64):
#         super(QNetwork, self).__init__()
#         self.fc1 = nn.Linear(state_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         self.fc3 = nn.Linear(hidden_dim, action_dim)
#     
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         return self.fc3(x)
#
# class ReplayBuffer:
#     def __init__(self, capacity=10000):
#         self.buffer = deque(maxlen=capacity)
#     
#     def push(self, state, action, reward, next_state, done):
#         self.buffer.append((state, action, reward, next_state, done))
#     
#     def sample(self, batch_size):
#         return random.sample(self.buffer, batch_size)
#     
#     def __len__(self):
#         return len(self.buffer)
#
# class DQNAgent:
#     def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99):
#         self.q_network = QNetwork(state_dim, action_dim)
#         self.target_network = QNetwork(state_dim, action_dim)
#         self.target_network.load_state_dict(self.q_network.state_dict())
#         
#         self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
#         self.gamma = gamma
#         self.replay_buffer = ReplayBuffer()
#         self.batch_size = 64
#         self.epsilon = 1.0
#         self.epsilon_decay = 0.995
#         self.epsilon_min = 0.01
#     
#     def get_action(self, state, training=True):
#         if training and random.random() < self.epsilon:
#             return random.randint(0, self.action_dim - 1)
#         else:
#             with torch.no_grad():
#                 state_tensor = torch.FloatTensor(state).unsqueeze(0)
#                 q_values = self.q_network(state_tensor)
#                 return q_values.argmax().item()
#     
#     def update(self):
#         if len(self.replay_buffer) < self.batch_size:
#             return
#         
#         # Sample batch
#         batch = self.replay_buffer.sample(self.batch_size)
#         states, actions, rewards, next_states, dones = zip(*batch)
#         
#         # Convert to tensors
#         states = torch.FloatTensor(states)
#         actions = torch.LongTensor(actions)
#         rewards = torch.FloatTensor(rewards)
#         next_states = torch.FloatTensor(next_states)
#         dones = torch.FloatTensor(dones)
#         
#         # Compute Q(s,a)
#         q_values = self.q_network(states).gather(1, actions.unsqueeze(1))
#         
#         # Compute target: r + gamma * max_a' Q_target(s', a')
#         with torch.no_grad():
#             next_q_values = self.target_network(next_states).max(1)[0]
#             targets = rewards + self.gamma * next_q_values * (1 - dones)
#         
#         # Loss and backprop
#         loss = nn.MSELoss()(q_values.squeeze(), targets)
#         self.optimizer.zero_grad()
#         loss.backward()
#         self.optimizer.step()
#         
#         # Decay epsilon
#         self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)
#     
#     def update_target_network(self):
#         self.target_network.load_state_dict(self.q_network.state_dict())

---

## References

### Books

1. **Shoham, Y., & Leyton-Brown, K.** (2008). *Multiagent Systems: Algorithmic, Game-Theoretic, and Logical Foundations*. Cambridge University Press.
   - Comprehensive coverage of game theory for multi-agent systems
   - Chapter 3: Strategic games and Nash equilibria

2. **Sutton, R. S., & Barto, A. G.** (2018). *Reinforcement Learning: An Introduction* (2nd ed.). MIT Press.
   - Definitive textbook on RL fundamentals
   - Free online: http://incompleteideas.net/book/the-book-2nd.html

3. **Busoniu, L., Babuska, R., & De Schutter, B.** (2008). *A Comprehensive Survey of Multiagent Reinforcement Learning*. IEEE Transactions on Systems, Man, and Cybernetics.
   - Survey of multi-agent RL techniques

### Papers - Prediction

4. **Lefèvre, S., Vasquez, D., & Laugier, C.** (2014). "A survey on motion prediction and risk assessment for intelligent vehicles." *ROBOMECH Journal*, 1(1), 1-14.
   - Excellent overview of prediction methods
   - Comparison of physics-based vs. learning-based approaches

5. **Deo, N., & Trivedi, M. M.** (2018). "Convolutional social pooling for vehicle trajectory prediction." *IEEE Conference on Computer Vision and Pattern Recognition Workshops*.
   - CNN-based approach for trajectory prediction
   - Captures spatial interactions between agents

6. **Alahi, A., et al.** (2016). "Social LSTM: Human trajectory prediction in crowded spaces." *IEEE Conference on Computer Vision and Pattern Recognition*.
   - LSTM with social pooling layer
   - Foundational paper for learning-based prediction

7. **Chai, Y., et al.** (2019). "MultiPath: Multiple Probabilistic Anchor Trajectory Hypotheses for Behavior Prediction." *Conference on Robot Learning*.
   - Multi-modal prediction with anchor trajectories
   - Used in Waymo's production system

8. **Cui, H., et al.** (2019). "Multimodal trajectory predictions for autonomous driving using deep convolutional networks." *IEEE International Conference on Robotics and Automation*.
   - Deep learning for multi-modal prediction
   - Includes uncertainty quantification

### Papers - Game Theory

9. **Schwarting, W., Alonso-Mora, J., & Rus, D.** (2018). "Planning and decision-making for autonomous vehicles." *Annual Review of Control, Robotics, and Autonomous Systems*, 1, 187-210.
   - Section on game-theoretic planning
   - Overview of interaction-aware decision making

10. **Fisac, J. F., et al.** (2019). "Hierarchical game-theoretic planning for autonomous vehicles." *IEEE International Conference on Robotics and Automation*.
    - Combines game theory with hierarchical planning
    - Addresses computational complexity

11. **Sadigh, D., et al.** (2016). "Planning for autonomous cars that leverage effects on human actions." *Robotics: Science and Systems*.
    - How AV actions influence human behavior
    - Game-theoretic framework for interaction

12. **Tian, R., et al.** (2020). "Game-theoretic modeling of traffic in unsignalized intersection network for autonomous vehicle control verification and validation." *IEEE Transactions on Intelligent Transportation Systems*.
    - Application to intersections
    - Validation methodology

### Papers - Reinforcement Learning

13. **Sallab, A. E., et al.** (2017). "Deep reinforcement learning framework for autonomous driving." *Electronic Imaging*, 2017(19), 70-76.
    - DQN for lane keeping and obstacle avoidance
    - Comparison with traditional methods

14. **Isele, D., Rahimi, R., Cosgun, A., Subramanian, K., & Fujimura, K.** (2018). "Navigating occluded intersections with autonomous vehicles using deep reinforcement learning." *IEEE International Conference on Robotics and Automation*.
    - RL for intersection navigation
    - Handles partial observability

15. **Leurent, E., & Mercat, J.** (2019). "Social attention for autonomous decision-making in dense traffic." *arXiv preprint arXiv:1911.12250*.
    - Attention mechanisms for multi-agent RL
    - Open-source implementation: highway-env

16. **Mirchevska, B., et al.** (2018). "High-level decision making for safe and reasonable autonomous lane changing using reinforcement learning." *21st International Conference on Intelligent Transportation Systems*.
    - Safe RL for lane changing
    - Incorporates safety constraints

17. **Tram, T., et al.** (2018). "Learning to communicate and correct pose errors." *Conference on Robot Learning*.
    - Multi-agent communication for coordination
    - Emergent communication protocols

### Papers - Integrated Approaches

18. **Hubmann, C., et al.** (2018). "Decision making for autonomous driving considering interaction and uncertain prediction of surrounding vehicles." *IEEE Intelligent Vehicles Symposium*.
    - Combines prediction and planning
    - POMDP formulation with interaction

19. **Sun, L., et al.** (2020). "Courteous autonomous cars." *IEEE/RSJ International Conference on Intelligent Robots and Systems*.
    - Social norms in autonomous driving
    - Politeness and assertiveness trade-offs

20. **Bouton, M., et al.** (2020). "Reinforcement learning with probabilistic guarantees for autonomous driving." *Conference on Robot Learning*.
    - Safe RL with formal guarantees
    - Shielding approach

### Datasets

21. **NGSIM** - Next Generation Simulation
    - https://ops.fhwa.dot.gov/trafficanalysistools/ngsim.htm
    - Highway trajectory data
    - Used for learning driver behavior models

22. **highD Dataset**
    - https://www.highd-dataset.com/
    - Drone-recorded highway trajectories
    - 110,000+ vehicle trajectories

23. **Argoverse**
    - https://www.argoverse.org/
    - Motion forecasting benchmark
    - HD maps + trajectory data

24. **nuScenes**
    - https://www.nuscenes.org/
    - Prediction challenge
    - Multi-modal sensor data

### Software & Tools

25. **SUMO** - Simulation of Urban MObility
    - https://www.eclipse.org/sumo/
    - Open-source traffic simulator
    - Supports multi-agent scenarios

26. **CARLA** - Open-source simulator for autonomous driving
    - http://carla.org/
    - Realistic environments
    - RL integration with OpenAI Gym

27. **Highway-Env**
    - https://github.com/eleurent/highway-env
    - Lightweight driving environments for RL
    - Fast prototyping

28. **SMARTS** - Scalable Multi-Agent RL Training School
    - https://github.com/huawei-noah/SMARTS
    - Multi-agent driving scenarios
    - Diverse agent policies

### Courses

29. **Stanford CS238: Decision Making under Uncertainty**
    - https://web.stanford.edu/class/aa228/
    - MDPs, POMDPs, game theory

30. **Berkeley CS285: Deep Reinforcement Learning**
    - http://rail.eecs.berkeley.edu/deeprlcourse/
    - Modern deep RL techniques
    - Policy gradients, Q-learning, actor-critic